# Аналитика по фильмам

## Подготовка:
Здесь мы импортируем свой класс Analytics и создаём его объект. При инициализации будут прочитаны все файлы в указанном  каталоге. Если с ними есть проблемы (файлов нет или они не того формата), возникнет соответствующее исключение.

In [31]:
from movielens_analysis import Analytics
analytics = Analytics('ml-latest-small')

Класс Analytics позволяет добраться до методов четырёх подклассов: Links, Movies, Ratings, Tags.

## Методы подкласса Links

### .get_imdb(list[, list])
Метод get_imdb() позволяет загрузить с сайта www.imdb.com данные о фильмах (указывается список ID) с помощью парсинга.
Посмотрим, какую информацию можно получить например по списку фильмов [317, 318, 319, 320].
Вторым агрументом может быть передан список параметров, но по умолчанию это ['Director', 'Budget', 'Cumulative Worldwide Gross', 'Runtime']

In [32]:
%timeit -n 1 -r 1 analytics.links.get_imdb([317, 318, 319, 320])
get_imdb = analytics.links.get_imdb([317, 318, 319, 320])
print(get_imdb)

4.27 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
[[320, 'N/A', 'N/A', '102780', '01:36:00'], [319, 'Danny Boyle', '2500000', '2079569', '01:29:00'], [318, 'Frank Darabont', '25000000', '28884504', '02:22:00'], [317, 'John Pasquin', '22000000', '190539357', '01:37:00']]


Как видно, 4 запроса заняли довольно большое время. При этом не все элементы удалось найти ('N/A').
Выводы:
1. Данные нужно создавать не одновременно с процессом анализа, а заранее. Желательно их положить в файл с быстрым доступом, а не каждый раз забирать по сети
2. Данные могут содержать ошибки и пустые места, это нужно учитывать при анализе.
Поскольку этот проект не позволяет менять подход, то чтобы делать дальнейший анализ, мы запросом только 10 фильмов в каждом методе.

### .top_directors(n)
Метод top_directors(n) позволяет посмотреть самых популярных режиссёров. Однако по 10 фильмам найти трёх самых популярных бесполезно - у каждого по 1 фильму ))

In [33]:
%timeit -n 1 -r 1 analytics.links.top_directors(1)
top_directors = analytics.links.top_directors(3)
print(top_directors)

11.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
{'Martin Campbell': 1, 'Peter Hyams': 1, 'Peter Hewitt': 1}


## .most_expensive(n)
Метод most_expensive(n) позволяет посмотреть N фильмов с самым большим бюджетом (не забываем, что выборка состоит только из 10 первых фильмов, а первыми в списке идут в основном старые фильмы):

In [34]:
%timeit -n 1 -r 1 analytics.links.most_expensive(1)
most_expensive = analytics.links.most_expensive(3)
print(most_expensive)

10.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
{'Jumanji (1995)': 65000000, 'GoldenEye (1995)': 60000000, 'Heat (1995)': 60000000}


Обратите внимание: здесь и далее названия файлов берутся из файла 'movies.csv'.

## .most_profitable(n)
Метод most_profitable(n) позволяет посмотреть N фильмов с самыми большими сборами:

In [35]:
%timeit -n 1 -r 1 analytics.links.most_profitable(1)
most_profitable = analytics.links.most_profitable(5)
print(most_profitable)

10.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
{'Toy Story (1995)': 364436586, 'GoldenEye (1995)': 292194034, 'Jumanji (1995)': 197821940, 'Heat (1995)': 127436818, 'Waiting to Exhale (1995)': 65452156}


## .longest(n))
Метод longest(n) позволяет посмотреть N фильмов из выборки с самым большой длительностью (длительность естественно тоже указана):

In [36]:
%timeit -n 1 -r 1 analytics.links.longest(1)
longest = analytics.links.longest(5)
print(longest)

10.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
{'Heat (1995)': '02:50:00', 'GoldenEye (1995)': '02:10:00', 'Sabrina (1995)': '02:07:00', 'Waiting to Exhale (1995)': '02:04:00', 'Sudden Death (1995)': '01:51:00'}


## .top_cost_per_minute(n))
Метод top_cost_per_minute(n) позволяет посмотреть N фильмов из выборки с самой большой ценой минуты (иногда фильмы длинные и дорогие, иногда короткие и дешёвые, этот параметр - "цена съёмки одной минуты"):

In [37]:
%timeit -n 1 -r 1 analytics.links.top_cost_per_minute(1)
top5costs = analytics.links.top_cost_per_minute(5)
print(top5costs)

10.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
{'Jumanji (1995)': 625000.0, 'GoldenEye (1995)': 461538.46153846156, 'Sabrina (1995)': 456692.91338582675, 'Toy Story (1995)': 370370.3703703704, 'Heat (1995)': 352941.17647058825}


Данные приведены в float-типе. Для удобства переведём их в int.
Нам запрешено использовать что-либо, кроме класса Analytics, поэтому мы предусмотрели в нём такой метод:

In [38]:
%timeit analytics.dict_to_int(top5costs)
dict_to_int = analytics.dict_to_int(top5costs)
print(dict_to_int)

442 ns ± 4.08 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
{'Jumanji (1995)': 625000, 'GoldenEye (1995)': 461538, 'Sabrina (1995)': 456692, 'Toy Story (1995)': 370370, 'Heat (1995)': 352941}


## Методы подкласса Movies
Подкласс Movies обрабатывает только файл 'movies.csv' (не обращается к сторонним сайтам). Поэтому работает заметно быстрее

## .dist_by_release()
Метод dist_by_release() выводит все найденные годы выпуска фильмов в порядке убывания количества фильмов, выпущенных в этот год. Можно сказать, что самими "активными" были нулевые годы - по 300 фильмов каждый год. И кака найти время, чтобы их все пересмотреть?

In [39]:
%timeit analytics.movies.dist_by_release()
dist_by_release = analytics.movies.dist_by_release()
print(dist_by_release)

5.36 ms ± 20.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
{'2002': 311, '2006': 295, '2001': 294, '2007': 284, '2000': 283, '2009': 282, '2003': 279, '2004': 279, '2014': 278, '1996': 276, '2015': 274, '2005': 273, '2008': 269, '1999': 263, '1997': 260, '1995': 259, '1998': 258, '2011': 254, '2010': 247, '2013': 239, '1994': 237, '2012': 233, '2016': 218, '1993': 198, '1992': 167, '1988': 165, '1987': 153, '1990': 147, '1991': 147, '2017': 147, '1989': 142, '1986': 139, '1985': 126, '1984': 101, '1981': 92, '1980': 89, '1982': 87, '1983': 83, '1979': 69, '1977': 63, '1973': 59, '1978': 59, '1965': 47, '1971': 47, '1974': 45, '1976': 44, '1964': 43, '1967': 42, '1968': 42, '1975': 42, '1966': 42, '2018': 41, '1962': 40, '1972': 39, '1963': 39, '1959': 37, '1960': 37, '1955': 36, '1969': 35, '1961': 34, '1970': 33, '1957': 33, '1958': 31, '1953': 30, '1956': 30, '1940': 25, '1949': 25, '1954': 23, '1942': 23, '1939': 23, '1946': 23, '1951': 22, '1950': 21, '1947': 20, '1948

## .dist_by_genres()
Метод dist_by_genres(n) делает то же самое с жанрами. Видно, что самими популярными жанрами являются Драма, Комадия и Триллер. Люди любят играть, смеяться и пугаться. Смеяться любят почти в 2 раза чаще, чем пугаться ))

In [40]:
%timeit analytics.movies.dist_by_genres()
dist_by_genres = analytics.movies.dist_by_genres()
print(dist_by_genres)

1.44 ms ± 5.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
{'Drama': 4361, 'Comedy': 3756, 'Thriller': 1894, 'Action': 1828, 'Romance': 1596, 'Adventure': 1263, 'Crime': 1199, 'Sci-Fi': 980, 'Horror': 978, 'Fantasy': 779, 'Children': 664, 'Animation': 611, 'Mystery': 573, 'Documentary': 440, 'War': 382, 'Musical': 334, 'Western': 167, 'IMAX': 158, 'Film-Noir': 87, '(no genres listed)': 34}


## .most_genres(n)
Метод most_genres(n) выводит N фильмов с самым большим количеством жанров.

In [41]:
%timeit analytics.movies.most_genres(3)
most_genres = analytics.movies.most_genres(3)
print(most_genres)

1.21 ms ± 7.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
{'Rubber (2010)': 10, 'Patlabor: The Movie (Kidô keisatsu patorebâ: The Movie) (1989)': 8, 'Mulan (1998)': 7}


Сразу 10 жанров в одном фильме. Вот это да! Интересно, какие? Чтобы посмотреть, можно воспользоваться дополнительной функцией

## .get_movies_from_title(list_of_titles)
Метод get_movies_from_title(list_of_titles) выводит словарь, в котором значениями указан список жанров каждого запрошенного фильма.
Это дополнительный метод, и на него тоже написаны тесты.

In [42]:
%timeit analytics.movies.get_movies_from_title(['Rubber (2010)', 'Mulan (1998)'])
get_movies_from_title = analytics.movies.get_movies_from_title(['Rubber (2010)', 'Mulan (1998)'])
print(get_movies_from_title)

390 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
{'Mulan (1998)': ['Adventure', 'Animation', 'Children', 'Comedy', 'Drama', 'Musical', 'Romance'], 'Rubber (2010)': ['Action', 'Adventure', 'Comedy', 'Crime', 'Drama', 'Film-Noir', 'Horror', 'Mystery', 'Thriller', 'Western']}


Вот такое кино - все жанры в одном фильме. Аж захотелось посмотреть.

## Analysing data from tags.csv

#### We already have an instance of Analytics class with specific data path.
Data file should have correct header, otherwise exception has been thrown.
From there, we will use Tags class attributes to work with tags data.

In [43]:
a = analytics

#### Les't look at tags with most words!

In [44]:
%timeit a.tags.most_words(5)
most_words = a.tags.most_words(5)
print(most_words)

316 µs ± 2.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
{'Something for everyone in this one... saw it without and plan on seeing it with kids!': 16, 'the catholic church is the most corrupt organization in history': 10, 'villain nonexistent or not needed for good story': 8, 'It was melodramatic and kind of dumb': 7, '06 Oscar Nominated Best Movie - Animation': 7}


Longest tag is 16 words long! Wow! Looking at 5 tags with most words we see that the longest is 16, then 10, 8, 7 and 7. That's quite a bit of words!

#### We have looked at most words, bu what about the number of characters in a tag? Will top 5 in terms of number of characters be the same or different?

In [45]:
%timeit a.tags.longest(5)
most_chars = a.tags.longest(5)
print(most_chars)

247 µs ± 3.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
['Something for everyone in this one... saw it without and plan on seeing it with kids!', 'the catholic church is the most corrupt organization in history', 'villain nonexistent or not needed for good story', 'r:disturbing violent content including rape', '06 Oscar Nominated Best Movie - Animation']


Actually, the same four with most words are the longest! Only one entry in the list is different from keys in the 'most_words' dictionary!

#### But a smart way to check the statement above is to use something called 'intersection' which will return to us information about entries that are in both lists!

In [46]:
%timeit a.tags.most_words_and_longest(5)
most_chars_and_words = a.tags.most_words_and_longest(5)
print(most_chars_and_words)

569 µs ± 3.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
['the catholic church is the most corrupt organization in history', 'villain nonexistent or not needed for good story', '06 Oscar Nominated Best Movie - Animation', 'Something for everyone in this one... saw it without and plan on seeing it with kids!']


Four items in the intersection list. This confirms our observations before, but now we can do it automatically!

#### Now let's look at popularity. Sure there are some tags that are more popular then others. Many users will tend to use a lot of the same tags.

In [47]:
%timeit a.tags.most_popular(5)
most_popular = a.tags.most_popular(5)
print(most_popular)

215 µs ± 1.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
{'In Netflix queue': 131, 'atmospheric': 36, 'superhero': 24, 'thought-provoking': 24, 'funny': 23}


Turns out, many people like to tag movies as being in their Netflix queue! It is the most popular tag. Other categories follow.

#### Now let's try to look for something specific.

In [48]:
%timeit a.tags.tags_with('none')
tags_with = a.tags.tags_with('bob')
print(tags_with)
tags_with = a.tags.tags_with('of')
print(tags_with)
tags_with = a.tags.tags_with('horror')
print(tags_with)

362 µs ± 2.52 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
['jay and silent bob']
['Guardians of the Galaxy', 'It was melodramatic and kind of dumb', 'Shakespeare sort of', 'Stones of Summer', 'ark of the covenant', 'end of the world', 'freedom of expression', 'lack of development', 'lack of plot', 'lack of story', 'lord of the rings', 'out of order', 'purity of essence', 'representation of children', 'start of a beautiful friendship']
['Horror', 'horror']


We found one tag containing word 'bob', all the tags containing 'of' and two versions of tags containing word 'horror'

## Analysing data from ratings.csv

#### Every rating in our data has an assosiated timestamp (Unix time). We can use this data to see when the ratings were given to movies.

In [49]:
%timeit a.ratings.movies.dist_by_year()
dist_by_year = a.ratings.movies.dist_by_year()
print(dist_by_year)

141 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
{'1996': 6040, '1997': 1916, '1998': 507, '1999': 2439, '2000': 10061, '2001': 3922, '2002': 3478, '2003': 4014, '2004': 3279, '2005': 5813, '2006': 4059, '2007': 7114, '2008': 4351, '2009': 4158, '2010': 2301, '2011': 1690, '2012': 4656, '2013': 1664, '2014': 1439, '2015': 6616, '2016': 6703, '2017': 8198, '2018': 6418}


We see that our ratings data has some ratings from 1996 till 2018.

#### Let's see how many different ratings our data has

In [50]:
%timeit a.ratings.movies.dist_by_rating()
dist_by_rating = a.ratings.movies.dist_by_rating()
print(dist_by_rating)

4.8 ms ± 247 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
{'0.5': 1370, '1.0': 2811, '1.5': 1791, '2.0': 7551, '2.5': 5550, '3.0': 20047, '3.5': 13136, '4.0': 26818, '4.5': 8551, '5.0': 13211}


So ratings are distributed in 0.5 increments from 0.5 to 5.0. The most ratings are 4.0 and 3.0.

#### Now let's see the most rated movies by the number of ratings.

In [51]:
%timeit a.ratings.movies.top_by_num_of_ratings(5)
top_by_num_of_ratings = a.ratings.movies.top_by_num_of_ratings(5)
print(top_by_num_of_ratings)

5.28 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
{'Forrest Gump (1994)': 329, '"Shawshank Redemption': 317, 'Pulp Fiction (1994)': 307, '"Silence of the Lambs': 279, '"Matrix': 278}


I think everyone should know these, no wonder they are the most rated!

#### The way we designed our classes allows us to ask the same question about the users: who has the most ratings?

In [52]:
%timeit a.ratings.users.top_by_num_of_ratings(5)
top_by_num_of_ratings = a.ratings.users.top_by_num_of_ratings(5)
print(top_by_num_of_ratings)

4.15 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
{'414': 2698, '599': 2478, '474': 2108, '448': 1864, '274': 1346}


Keys in this dict are user ids. We don't have our users names, but now we know that one person left 2698 ratings for movies and the second place is not far behind!

#### Now let's see top movies by ratings. We can do this both using mean(average) and median values

In [53]:
%timeit a.ratings.movies.top_by_ratings(5, metric='average')
top_by_ratings = a.ratings.movies.top_by_ratings(5, metric='average')
print(top_by_ratings)

27.2 ms ± 242 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
{'The Jinx: The Life and Deaths of Robert Durst (2015)': 5.0, 'Galaxy of Terror (Quest) (1981)': 5.0, 'Alien Contamination (1980)': 5.0, "I'm the One That I Want (2000)": 5.0, 'Lesson Faust (1994)': 5.0}


Now the same, but using median metric. This is not so different for top5 movies.

In [54]:
%timeit a.ratings.movies.top_by_ratings(5, metric='median')
top_by_ratings = a.ratings.movies.top_by_ratings(5, metric='median')
print(top_by_ratings)

25.3 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
{'The Jinx: The Life and Deaths of Robert Durst (2015)': 5.0, 'Galaxy of Terror (Quest) (1981)': 5.0, 'Alien Contamination (1980)': 5.0, 'Troll 2 (1990)': 5.0, "I'm the One That I Want (2000)": 5.0}


Using the same method on users, we can find those who only give good ratings to movies.
Users are represented as their ids.

In [55]:
%timeit a.ratings.users.top_by_ratings(5, metric='average')
top_by_ratings = a.ratings.users.top_by_ratings(5, metric='average')
print(top_by_ratings)

18.9 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
{'53': 5.0, '251': 4.87, '515': 4.85, '25': 4.81, '30': 4.74}


Now we know that user with id 53 only gave 5 star ratings.

#### If we want, we can see what movies are the most controversial ones. 
This is calculated by variance of ratings. variance() is a function of a built-in 'statistics' module.
Today, we can't import this module, so we built our own variance function:

In [56]:
%timeit a.ratings.movies.top_controversial(5)
top_controversial = a.ratings.movies.top_controversial(5)
print(top_controversial)

70.4 ms ± 381 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
{'Pulp Fiction (1994)': 277.33, '"Matrix': 263.46, 'Forrest Gump (1994)': 226.64, 'Braveheart (1995)': 221.01, "Schindler's List (1993)": 208.62}


The higher the value, the more controversial in terms of ratings the movie is.
Not surprisingly, movies like 'Pulp Fiction' and 'Schindler's List' cause a lot of controversy.

Using the same method on users we can see those with the most variance in ratings, but this is not very meaningful for us as we study movies.

#### Finalizing this report, we can say that we took a pretty big dive into movielens dataset and learned quiet a lot about movies, having fun in the process!